[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

## Objetivo.

El objetivo de este capítulo es ejemplificar el uso de herramientas como *NLTK* y *networkX* en un ejemplo práctico.

EL código siguiente definir;a algunas funciones capaces de:

* Conectarse a la API de Twitter.
* Realizar búsquedas a partir de un términoy obtener el resultado en formato JSON.
* Guardar y recuperar en un archivo JSON diversos tuits.
* Crear una grafo que relaciona a usuarios con sus tuits.
* Normalizar los textos de los tuits.
* Crear una nube de palabras a partir de los textos de los tuis.

## Operaciones iniciales.

* Se instalarán los compomentes.

In [ ]:
!pip install nltk networkx twitter wordcloud matplotlib

* Se importarán las herramientas necesarias para ejecutar el código.

In [ ]:
import csv, twitter, json, nltk
from functools import reduce
import networkx as nx
from matplotlib import pyplot as plt
from wordcloud import WordCloud

* Se inicializarán las credenciales.

In [ ]:
API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_SECRET = "", "", "", ""

## La función *acceede_a_tw()*.

* Extraerá las credenciales de la API de Twitter a partir de un achivo cuya ruta es definida como argumento.
* Regresará un objeto que contiene una conexión a la API de Twitter.

In [ ]:
def accede_a_tw(fuente):
    (API_KEY,
     API_SECRET,
     ACCESS_TOKEN,
     ACCESS_SECRET) = open(
            fuente, 'r').read().splitlines()
    auth = twitter.oauth.OAuth(ACCESS_TOKEN,
                           ACCESS_SECRET,
                           API_KEY,
                           API_SECRET)
    return twitter.Twitter(auth=auth)

## La función *busqueda_tw()*.

Esta función realizará la búsqueda de un término mediante la API de Twitter.

In [ ]:
def busqueda_tw(tw, termino):
    return tw.search.tweets(q=termino, lang="es", count="500")["statuses"]

In [ ]:
def carga_lista(archivo):
    try:
        with open(archivo, "r") as f:
            lista = [elemento.replace(" ", "") for elemento in \
                     reduce(lambda x, y: x + y,
                            [linea for linea in csv.reader(f, dialect="unix")])]
    except IOError:
        lista = []
    return lista

In [ ]:
def carga_tuits(archivo):
    try:
        f = open(archivo, "r")
        resultado = json.load(f)
    except IOError:
        resultado = []
    else:
        f.close()
    return resultado

In [ ]:
def guarda_tuits(tuits, archivo):
    with open(archivo, "w") as f:
        json.dump(tuits, f, indent=1)

In [ ]:
def mezcla_tuits(actuales, nuevos):
    for tuit in nuevos: 
        if tuit["id"] not in [actual["id"] for actual in actuales]:
            actuales.append(tuit)
    return actuales

In [ ]:
def limpiar(texto):
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    limpio = tokenizer.tokenize(texto)    
    return limpio

In [ ]:
def analiza_menciones(tuits):
    pares = []
    nodos = []
    for tuit in tuits:
        usuario = tuit["user"]["screen_name"]
        nodos.append(usuario)
        menciones = [mencion["screen_name"] for mencion in tuit["entities"]["user_mentions"]]
        for mencion in menciones:
            if mencion != [] and usuario != mencion:
                par = (usuario, mencion)
                pares.append(par)
    nodos = list(set(nodos))
    pares = list(set(pares))
    G = nx.Graph()
    G.add_nodes_from(nodos)
    G.add_edges_from(pares)
    plt.figure(figsize=(32,32))
    nx.draw_networkx(G)

In [ ]:
def refina_texto(tuits, lista, termino):
    lista_negra = carga_lista(lista) + [palabra.replace("@", "") for palabra in termino.split()]
    texto =""
    for i in range(0, len(lista_negra)):
        lista_negra[i] = lista_negra[i].lower()
    for tuit in tuits:
        texto += (tuit["text"] + " ")
        depurador = nltk.RegexpTokenizer(r'\w+')
    limpio = depurador.tokenize(texto)
    texto = ""
    for termino in limpio:
        termino = termino.lower()
        if  termino not in lista_negra:
            texto += (termino + " ") 
    return str(texto)

In [ ]:
def nube(texto):
    wordcloud = WordCloud().generate(texto)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
def main(archivo="tuits.json", lista="data/lista_negra.csv"): 
    termino = input("Término de búsqueda: ")
    tuits_previos = carga_tuits(archivo)
    tw = accede_a_tw("data/credenciales.txt")
    tuits_recientes = busqueda_tw(tw, termino)
    tuits = mezcla_tuits(tuits_previos, tuits_recientes)
    guarda_tuits(tuits, archivo)
    analiza_menciones(tuits)
    return refina_texto(tuits, lista, termino)

In [ ]:
texto = main()

In [ ]:
nube(texto)

In [ ]:
tuits = carga_tuits("tuits.json")
texto = refina_texto(tuits, "data/lista_negra.csv", "")

In [ ]:
nube(texto)

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>